In [26]:
import sqlite3
import pandas as pd

In [27]:
csv_data = pd.read_csv("output_norbert.csv")
pd.to_datetime(csv_data['date_key'])
pd.options.display.min_rows = 40
pd.options.display.max_rows = 40
pd.options.display.max_columns = 99
pd.options.display.max_colwidth = 25
pd.options.display.width = 9999

In [28]:
con = sqlite3.connect(":memory:")

In [29]:
csv_data.to_sql("users", con, if_exists="replace")

43167

In [34]:
query1 = """

WITH table1_temp AS (
SELECT
account_id,
date_key,
revenue,
LEAD(date_key, 1) OVER (PARTITION BY account_id ORDER BY date_key) as lead_1_month,
LEAD(date_key, 3) OVER (PARTITION BY account_id ORDER BY date_key) as lead_3_month,
LEAD(date_key, 6) OVER (PARTITION BY account_id ORDER BY date_key) as lead_6_month,
LEAD(date_key, 12) OVER (PARTITION BY account_id ORDER BY date_key) as lead_12_month
FROM users
WHERE 1=1
AND account_id = '5edf63c9-a4de-4592-b70d-8fa137c00f88'
),

table2_temp AS (
SELECT
account_id,
date_key,
revenue,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_1_month))/(365/12),0) as lead_1_mo_diff,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_3_month))/(365/12),0) as lead_3_mo_diff,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_6_month))/(365/12),0) as lead_6_mo_diff,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_12_month))/(365/12),0) as lead_12_mo_diff
FROM table1_temp
),

table3_temp AS (
SELECT
date_key,
COUNT(account_id) AS c_cur,
COUNT(CASE WHEN lead_1_mo_diff=-1.0 THEN account_id ELSE NULL END) AS c_1m,
COUNT(CASE WHEN lead_3_mo_diff=-3.0 THEN account_id ELSE NULL END) AS c_3m,
COUNT(CASE WHEN lead_6_mo_diff=-6.0 THEN account_id ELSE NULL END) AS c_6m,
COUNT(CASE WHEN lead_12_mo_diff=-12.0 THEN account_id ELSE NULL END) AS c_12m
FROM table2_temp
GROUP BY 1
)

SELECT date_key, lead_1_month, lead_3_month, lead_6_month, lead_12_month FROM table1_temp

"""

query2 = """
SELECT * FROM users where account_id = '5edf63c9-a4de-4592-b70d-8fa137c00f88'
"""

query3 = """
WITH calendar AS (
SELECT
DISTINCT(date_key)
FROM users
ORDER BY date_key
),

table1_temp AS (
SELECT
account_id,
date_key,
revenue,
LEAD(date_key, 1) OVER (ORDER BY date_key) as lead_1_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 1) OVER (ORDER BY date_key)))/(365/12),0), 0.0) as lead_1_mo_diff,
LEAD(date_key, 3) OVER (PARTITION BY account_id ORDER BY date_key) as lead_3_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 3) OVER (PARTITION BY account_id ORDER BY date_key)))/(365/12),0), 0.0) as lead_3_mo_diff,
LEAD(date_key, 6) OVER (PARTITION BY account_id ORDER BY date_key) as lead_6_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 6) OVER (PARTITION BY account_id ORDER BY date_key)))/(365/12),0), 0.0) as lead_6_mo_diff,
LEAD(date_key, 12) OVER (PARTITION BY account_id ORDER BY date_key) as lead_12_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 12) OVER (PARTITION BY account_id ORDER BY date_key)))/(365/12),0), 0.0) as lead_12_mo_diff
FROM users
WHERE 1=1
AND account_id IN ('5edf63c9-a4de-4592-b70d-8fa137c00f88', 'b41abbf3-ca43-48cd-8e8d-1aad0729bae6')
AND date_key BETWEEN '2019-01-01' AND '2019-12-31'
),

table2_temp AS (
SELECT
account_id,
date_key,
revenue,
lead_1_month,
lead_1_mo_diff,
SUM(CASE WHEN lead_1_mo_diff = -1.0 THEN 1 ELSE 0 END) as c_1m,
lead_3_month,
lead_3_mo_diff,
SUM(CASE WHEN lead_3_mo_diff = -3.0 THEN 1 ELSE 0 END) as c_3m
--date_key,
--COUNT(account_id) AS c_cur,
--COUNT(CASE WHEN lead_1_mo_diff=-1.0 THEN account_id ELSE NULL END) AS c_1m,
--COUNT(CASE WHEN lead_3_mo_diff=-3.0 THEN account_id ELSE NULL END) AS c_3m,
--COUNT(CASE WHEN lead_6_mo_diff=-6.0 THEN account_id ELSE NULL END) AS c_6m,
--COUNT(CASE WHEN lead_12_mo_diff=-12.0 THEN account_id ELSE NULL END) AS c_12m
FROM table1_temp
GROUP BY
account_id,
date_key,
revenue,
lead_1_month,
lead_1_mo_diff,
lead_3_month,
lead_3_mo_diff
)

-- SELECT * FROM calendar


SELECT
account_id,
date_key,
revenue,
lead_1_month,
lead_1_mo_diff,
--SUM(c_1m),
lead_3_month,
lead_3_mo_diff
--SUM(c_3m)
FROM table1_temp
ORDER BY 
date_key

"""

query4 = """
WITH calendar AS (
SELECT
date_key,
LEAD(date_key, 3) OVER (ORDER BY date_key) as date_key3,
LEAD(date_key, 6) OVER (ORDER BY date_key) as date_key6,
LEAD(date_key, 12) OVER (ORDER BY date_key) as date_key12
FROM users
WHERE 1=1
AND date_key > '2019-06-01'
GROUP BY date_key
ORDER BY date_key
),

fact_table_curr AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

fact_table_3m AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

fact_table_6m AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

fact_table_12m AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

all_joined AS (
SELECT
calendar.date_key as c_date_key,
fact_table_curr.date_key as f_curr_date_key,
SUBSTR(fact_table_curr.account_id,1,3) as f_curr_acc_id,
fact_table_curr.revenue as f_curr_rev,
calendar.date_key3 as c_date_key3,
fact_table_3m.date_key as f_3m_date_key,
SUBSTR(fact_table_3m.account_id,1,3) as f_3m_acc_id,
fact_table_6m.revenue as f_6m_rev,
calendar.date_key6 as c_date_key6,
fact_table_6m.date_key as f_6m_date_key,
SUBSTR(fact_table_6m.account_id,1,3) as f_6m_acc_id,
fact_table_6m.revenue as f_6m_rev,
calendar.date_key12 as c_date_key12,
fact_table_12m.date_key as f_12m_date_key,
SUBSTR(fact_table_12m.account_id,1,3) as f_12m_acc_id,
fact_table_12m.revenue as f_12m_rev
FROM calendar
LEFT JOIN fact_table_curr ON calendar.date_key = fact_table_curr.date_key
LEFT JOIN fact_table_3m ON (fact_table_curr.account_id = fact_table_3m.account_id AND calendar.date_key3 = fact_table_3m.date_key)
LEFT JOIN fact_table_6m ON (fact_table_curr.account_id = fact_table_6m.account_id AND calendar.date_key6 = fact_table_6m.date_key)
LEFT JOIN fact_table_12m ON (fact_table_curr.account_id = fact_table_12m.account_id AND calendar.date_key12 = fact_table_12m.date_key)
)

SELECT
c_date_key,
COUNT(f_curr_acc_id) as count_acc_curr,
COUNT(f_3m_acc_id) as count_acc_3m,
COUNT(f_6m_acc_id) as count_acc_6m,
COUNT(f_12m_acc_id) as count_acc_12m
FROM all_joined
WHERE 1=1
GROUP BY
c_date_key
ORDER BY
c_date_key

"""

query4_verif = """
WITH calendar AS (
SELECT
date_key,
LEAD(date_key, 1) OVER () as date_key1,
LEAD(date_key, 3) OVER () as date_key3,
LEAD(date_key, 6) OVER () as date_key6
--LEAD(date_key, 12) OVER () as date_key12
FROM users
WHERE 1=1
AND date_key BETWEEN '2019-01-01' AND '2019-07-01'
GROUP BY date_key
ORDER BY date_key
),

fact_table1 AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
AND date_key BETWEEN '2019-01-01' AND '2019-12-01'
AND account_id IN (
    'eba9dbd2-ff64-46cb-9ab8-08ddcc99485f',
    'c51272a2-28ad-4fa8-80c1-5b7566c439a8',
    '5a316a69-9d5e-4bdc-a908-8a109ed7f4ac',
    '17516fd3-55e1-4ebb-8050-2da61969976e',
    'b2ec20fc-27d1-43cf-83a3-e0ada52d0639'
    )
),

fact_table2 AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
AND date_key BETWEEN '2019-01-01' AND '2019-12-01'
AND account_id IN (
    'eba9dbd2-ff64-46cb-9ab8-08ddcc99485f',
    'c51272a2-28ad-4fa8-80c1-5b7566c439a8',
    '5a316a69-9d5e-4bdc-a908-8a109ed7f4ac',
    '17516fd3-55e1-4ebb-8050-2da61969976e',
    'b2ec20fc-27d1-43cf-83a3-e0ada52d0639'
    )
),

all_joined AS (
SELECT
calendar.date_key as c_date_key,
fact_table1.date_key as f1_date_key,
SUBSTR(fact_table1.account_id,1,3) as f1_acc_id,
fact_table1.revenue as f1_rev,
calendar.date_key3 as c_date_key3,
fact_table2.date_key as f2_date_key,
SUBSTR(fact_table2.account_id,1,3) as f2_acc_id,
fact_table2.revenue as f2_rev
FROM calendar
LEFT JOIN fact_table1 ON calendar.date_key = fact_table1.date_key
LEFT JOIN fact_table2 ON fact_table1.account_id = fact_table2.account_id AND calendar.date_key3 = fact_table2.date_key
)

SELECT
*
FROM all_joined
WHERE 1=1


"""

query4_presto = """ 

WITH calendar AS (
SELECT
date_key,
LEAD(date_key, 3) OVER (ORDER BY date_key) as date_key3,
LEAD(date_key, 6) OVER (ORDER BY date_key) as date_key6,
LEAD(date_key, 12) OVER (ORDER BY date_key) as date_key12
FROM clv.agg_acc_rev_monthly
WHERE date_key BETWEEN '2019-01-31' AND '2019-06-30'
GROUP BY date_key
ORDER BY date_key
),

fact_table_curr AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

fact_table_3m AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

fact_table_6m AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

fact_table_12m AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

all_joined AS (
SELECT
calendar.date_key as c_date_key,
fact_table_curr.date_key as f_curr_date_key,
fact_table_curr.account_id as f_curr_acc_id,
fact_table_curr.revenue as f_curr_rev,
calendar.date_key3 as c_date_key3,
fact_table_3m.date_key as f_3m_date_key,
fact_table_3m.account_id as f_3m_acc_id,
fact_table_3m.revenue as f_3m_rev
--calendar.date_key6 as c_date_key6,
--fact_table_6m.date_key as f_6m_date_key,
--fact_table_6m.account_id as f_6m_acc_id,
--fact_table_6m.revenue as f_6m_rev,
--calendar.date_key12 as c_date_key12,
--fact_table_12m.date_key as f_12m_date_key,
--fact_table_12m.account_id as f_12m_acc_id,
--fact_table_12m.revenue as f_12m_rev
FROM calendar
LEFT JOIN fact_table_curr ON calendar.date_key = fact_table_curr.date_key
LEFT JOIN fact_table_3m ON (fact_table_curr.account_id = fact_table_3m.account_id AND calendar.date_key3 = fact_table_3m.date_key)
--LEFT JOIN fact_table_6m ON (fact_table_curr.account_id = fact_table_6m.account_id AND calendar.date_key6 = fact_table_6m.date_key)
--LEFT JOIN fact_table_12m ON (fact_table_curr.account_id = fact_table_12m.account_id AND calendar.date_key12 = fact_table_12m.date_key)
)

SELECT
c_date_key,
COUNT(f_curr_acc_id) as count_acc_curr,
COUNT(f_3m_acc_id) as count_acc_3m
FROM all_joined
WHERE 1=1
GROUP BY
c_date_key
ORDER BY
c_date_key

"""

query5 = """ 
WITH calendar AS (
    SELECT 
        date_key
    FROM users
    WHERE date_key >= '2019-01-01'
    GROUP BY date_key
    ORDER BY date_key
),
distinct_accounts AS (
    SELECT
        account_id
    FROM users
    WHERE date_key >= '2019-01-01'
        AND revenue IS NOT NULL
        AND revenue > 0
    GROUP BY account_id
    ORDER BY account_id
),
distinct_sources AS (
    SELECT 
        source 
    FROM users 
    WHERE source = 'display_ads'
    GROUP BY source
    ORDER BY source
),
fact AS (
    SELECT
        date_key,
        account_id,
        revenue,
        source,
        date_key || account_id || source AS join_key
    FROM users
    WHERE source = 'display_ads'
        AND date_key >= '2019-01-01'
        AND revenue IS NOT NULL
        AND revenue > 0
),
crossed_joined AS (
    SELECT 
        date_key,
        account_id,
        source,
        date_key || account_id || source AS join_key 
    FROM calendar 
    CROSS JOIN distinct_accounts 
    CROSS JOIN distinct_sources
    WHERE join_key NOT IN (
        SELECT join_key FROM fact
    )
),
unioned AS (
    SELECT 
        date_key,
        account_id,
        revenue,
        source 
    FROM fact 
    UNION 
    SELECT
        date_key,
        account_id,
        0 AS revenue,
        source
    FROM crossed_joined
)

SELECT * FROM unioned ORDER BY account_id

"""




In [35]:
cohort_df = pd.read_sql_query(query5, con)
print(cohort_df)
cohort_df.to_sql("users_2", con, index=False, if_exists="replace")

         date_key                account_id   revenue       source
0      2019-01-01  00a9e301-7652-48e2-9c...  6.023267  display_ads
1      2019-02-01  00a9e301-7652-48e2-9c...  2.894099  display_ads
2      2019-03-01  00a9e301-7652-48e2-9c...  7.416970  display_ads
3      2019-04-01  00a9e301-7652-48e2-9c...  7.410220  display_ads
4      2019-05-01  00a9e301-7652-48e2-9c...  0.263766  display_ads
5      2019-06-01  00a9e301-7652-48e2-9c...  9.898501  display_ads
6      2019-07-01  00a9e301-7652-48e2-9c...  0.000000  display_ads
7      2019-08-01  00a9e301-7652-48e2-9c...  0.000000  display_ads
8      2019-09-01  00a9e301-7652-48e2-9c...  1.945312  display_ads
9      2019-10-01  00a9e301-7652-48e2-9c...  4.281214  display_ads
10     2019-11-01  00a9e301-7652-48e2-9c...  9.562664  display_ads
11     2019-12-01  00a9e301-7652-48e2-9c...  3.568451  display_ads
12     2020-01-01  00a9e301-7652-48e2-9c...  0.000000  display_ads
13     2020-02-01  00a9e301-7652-48e2-9c...  1.438361  display

24000

In [38]:
query5_presto = """ 

DROP TABLE clv.temp_rev_with_0_display_ads

CREATE TABLE IF NOT EXISTS clv.temp_rev_with_0_display_ads AS
WITH clv_table AS (
    SELECT
     a.*
    FROM clv.agg_acc_rev_monthly a
        LEFT JOIN roku.dim_account_detail b on a.account_id = b.account_id
    WHERE b.channel_store_code = 'us'
),
calendar AS (
    SELECT
        date_key
    FROM clv_table
    WHERE date_key >= '2019-01-01'
    GROUP BY date_key
    ORDER BY date_key
),
distinct_accounts AS (
    SELECT
        account_id
    FROM clv_table
    WHERE date_key >= '2019-01-01'
        AND hr IS NOT NULL
        AND hr > 0
    GROUP BY account_id
    ORDER BY account_id
),
distinct_sources AS (
    SELECT
        src
    FROM clv_table
    WHERE src = 'display_ads'
    GROUP BY src
    ORDER BY src
),
fact AS (
    SELECT
        date_key,
        account_id,
        src,
        hr,
        concat(date_key, account_id, src) AS join_key 
    FROM clv_table
        AND date_key >= '2019-01-01'
        AND src = 'display_ads'
        AND hr IS NOT NULL
        AND hr > 0
),
crossed_joined AS (
    SELECT
        date_key,
        account_id,
        src,
        concat(date_key, account_id, src) AS join_key
    FROM calendar
        CROSS JOIN distinct_accounts
        CROSS JOIN distinct_sources
    WHERE concat(date_key, account_id, src) NOT IN (
        SELECT join_key FROM fact
    )
),
unioned AS (
    SELECT 
        date_key,
        account_id,
        src,
        hr 
    FROM fact 
    UNION
    SELECT 
        date_key,
        account_id,
        src,
        0 AS hr
    FROM crossed_joined
)

SELECT * FROM unioned ORDER BY account_id

"""

query6_presto = """ 
WITH 
first_monetization_dates AS (
    SELECT 
        account_id,
        MIN(date_key) as first_monetization_date
    FROM clv.temp_rev_with_0_display_ads
    WHERE hr > 0 
    GROUP BY account_id
    ),
cohort AS (
    SELECT 
        account_id
    FROM first_monetization_dates
    WHERE first_monetization_date = '2020-01-31'
    ),
inactivity_table AS (
    SELECT
        a.account_id,
        a.date_key,
        a.hr,
        CASE WHEN (hr > 0) THEN 0 ELSE 1 END as inactive
    FROM clv.temp_rev_with_0_display_ads a
    RIGHT JOIN cohort on a.account_id = cohort.account_id
    WHERE a.date_key >= '2020-01-31'
    ORDER BY
    account_id,
    a.date_key asc
),
fact AS (
    SELECT 
        account_id,
        date_key,
        hr,
        inactive,
        SUM(gap) OVER (PARTITION BY account_id ORDER BY date_key) status_change
    FROM (
        SELECT
            account_id,
            date_key,
            hr,
            inactive,
            CASE WHEN inactive = LAG(inactive) OVER (PARTITION BY account_id ORDER BY date_key) THEN 0 ELSE 1 END gap
        FROM inactivity_table
    )
),
fact_2 AS (
    SELECT 
        *,
        SUM(inactive) OVER (PARTITION BY CONCAT(account_id, CAST(status_change AS VARCHAR)) ORDER BY date_key) AS cons_inactive
    FROM fact
    ORDER BY account_id, date_key
),
final AS (
    SELECT 
        *,
        CASE WHEN cons_inactive >= 6 THEN 0 ELSE 1 END AS not_churned
    FROM 
    fact_2
)

SELECT
    date_key,
    SUM(not_churned) AS ch1M
FROM final
GROUP BY date_key
ORDER BY date_key


"""

query6 = """ 
select * from users_2

"""

In [39]:
users_df = pd.read_sql_query(query6, con)
print(users_df)

         date_key                account_id   revenue       source
0      2019-01-01  00a9e301-7652-48e2-9c...  6.023267  display_ads
1      2019-02-01  00a9e301-7652-48e2-9c...  2.894099  display_ads
2      2019-03-01  00a9e301-7652-48e2-9c...  7.416970  display_ads
3      2019-04-01  00a9e301-7652-48e2-9c...  7.410220  display_ads
4      2019-05-01  00a9e301-7652-48e2-9c...  0.263766  display_ads
5      2019-06-01  00a9e301-7652-48e2-9c...  9.898501  display_ads
6      2019-07-01  00a9e301-7652-48e2-9c...  0.000000  display_ads
7      2019-08-01  00a9e301-7652-48e2-9c...  0.000000  display_ads
8      2019-09-01  00a9e301-7652-48e2-9c...  1.945312  display_ads
9      2019-10-01  00a9e301-7652-48e2-9c...  4.281214  display_ads
10     2019-11-01  00a9e301-7652-48e2-9c...  9.562664  display_ads
11     2019-12-01  00a9e301-7652-48e2-9c...  3.568451  display_ads
12     2020-01-01  00a9e301-7652-48e2-9c...  0.000000  display_ads
13     2020-02-01  00a9e301-7652-48e2-9c...  1.438361  display